In [1]:
import os
import time
import shutil
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import keras
import gc
from PIL import Image
%matplotlib inline
from keras.models import Sequential,Model
from tensorflow.keras.applications import ResNet50, ResNet50V2, DenseNet201
from keras.layers import Dense, Activation, Dropout
import keras.backend as K
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.layers import Conv2D, MaxPool2D, AvgPool2D, Flatten, GlobalAveragePooling2D, AveragePooling2D
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report, cohen_kappa_score, roc_auc_score

In [2]:
# img_size = (768, 768)
# optimizer = SGD(learning_rate=5e-5, momentum=0.9)
# # optimizer = Adam(learning_rate=1e-5)
# foldId = 0

img_size = (512,512)
batch_size = 16
# optimizer = SGD(learning_rate=5e-5, momentum=0.9)
optimizer = Adam(learning_rate=1e-4)
foldId = 0

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_df = pd.read_csv("../input/diabetic-retinopathy-preprocessed-data/trainLabels.csv")

indexes = []
for i in range(len(train_df['image'])):
    if train_df['kfold'][i] == foldId:
        indexes.append(i)
train_df.drop(indexes, axis=0, inplace=True)

train_df['image'] = [str(x)+'.jpeg' for x in train_df['image']]
train_df['level'] = [str(x) for x in train_df['level']]
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory="../input/diabetic-retinopathy-preprocessed-data/Data/Data/train",
    x_col="image",
    y_col='level',
    subset="training",
    batch_size=4,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=img_size)

Found 25210 validated image filenames belonging to 5 classes.


In [4]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_df = pd.read_csv("../input/diabetic-retinopathy-preprocessed-data/trainLabels.csv")
indexes = []
for i in range(len(val_df['image'])):
    if val_df['kfold'][i] != foldId:
        indexes.append(i)
val_df.drop(indexes, axis=0, inplace=True)

val_df['image'] = [str(x)+'.jpeg' for x in val_df['image']]
val_df['level'] = [str(x) for x in val_df['level']]
val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory="../input/diabetic-retinopathy-preprocessed-data/Data/Data/train",
    x_col="image",
    y_col='level',
    batch_size=4,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=img_size)

Found 8404 validated image filenames belonging to 5 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_df = pd.read_csv("../input/diabetic-retinopathy-preprocessed-data/Data/Data/test_labels.csv")
test_df['image'] = [str(x)+'.jpeg' for x in test_df['image']]
test_df['level'] = [str(x) for x in test_df['level']]
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory="../input/diabetic-retinopathy-preprocessed-data/Data/Data/test",
    x_col="image",
    y_col='level',
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    target_size=img_size)

Found 53574 validated image filenames belonging to 5 classes.


/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 2 invalid image filename(s) in x_col="image". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [6]:
denseNet = DenseNet201(include_top=False, weights='imagenet', input_shape=(img_size[0], img_size[1], 3))
output = denseNet.layers[-1].output
output = keras.layers.Flatten()(output)
denseNet = Model(denseNet.input, output)

model = Sequential()
model.add(denseNet)
model.add(Dropout(0.3))
model.add(Dense(5, activation='softmax'))

model.compile(optimizer, 
            loss='categorical_crossentropy',
            metrics=['accuracy'])

checkpoint_filepath = "./trainedmodel.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( filepath=checkpoint_filepath,
                          save_weights_only=False,
                          monitor='val_loss',
                          mode='auto',
                          save_best_only=True)

early_stopping = EarlyStopping(
                monitor="val_loss",
                patience=3,
                verbose=1,
                mode="auto",
)

model.summary()

2022-07-12 22:31:28.750611: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-12 22:31:28.866082: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-12 22:31:28.866943: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-12 22:31:28.868197: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

74850304/74836368 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Functional)           (None, 491520)            18321984  
_________________________________________________________________
dropout (Dropout)            (None, 491520)            0         
_________________________________________________________________
dense (Dense)                (None, 5)                 2457605   
Total params: 20,779,589
Trainable params: 20,550,533
Non-trainable params: 229,056
_________________________________________________________________


In [7]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size

In [8]:
# model.fit_generator(generator=train_generator,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_data=val_generator,
#                     validation_steps=STEP_SIZE_VALID,
#                     epochs=25,
#                     callbacks=[model_checkpoint_callback, early_stopping])

In [9]:
model.load_weights("../input/diabetic-retinopathy-densenet201-model-file/trainedmodel.h5")

In [10]:
test_generator.reset()
pred=model.predict_generator(test_generator,
                            steps=STEP_SIZE_TEST,
                            verbose=1)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
2022-07-12 22:31:39.956714: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-12 22:31:43.598153: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


53574/53574 [==============================] - 2351s 44ms/step


In [11]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())

predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames

predcitionsMap = {filenames[i]: predictions[i] for i in range(len(filenames))}

In [12]:
from collections import Counter
P = Counter(predictions)
T = Counter(test_df['level'])
print(sorted(P.items()))
print(sorted(T.items()))

correct = 0
predLabels = []
indexes = []
for i in range(len(test_df['image'])):
    if test_df['image'][i] in predcitionsMap:
        predLabels.append(predcitionsMap[test_df['image'][i]])
        if predcitionsMap[test_df['image'][i]] == test_df['level'][i]:
            correct += 1
    else:
        predLabels.append("NA")
        indexes.append(i)
acc = (correct / len(test_df['level'])) * 100
print(acc)

test_df['predLevel'] = predLabels
test_df.drop(indexes, axis=0, inplace=True)
# test_df.to_csv("./InceptionV3AndInceptionResNetV2predictions.csv", index=False)

print(classification_report(test_df['level'], test_df['predLevel']))

print(confusion_matrix(test_df["level"], test_df["predLevel"]))

print("Test Accuracy score : ", accuracy_score(test_df['level'], test_df['predLevel']))
print("Test Weighted F1 score : ", f1_score(test_df['level'], test_df['predLevel'], average="weighted"))
print("Test Micro F1 score : ", f1_score(test_df['level'], test_df['predLevel'], average="micro"))
print("Test Precision score : ", precision_score(test_df['level'], test_df['predLevel'], average="weighted"))
print("Test Recall score : ", recall_score(test_df['level'], test_df['predLevel'], average="weighted"))
print("Test Cohen Kappa Score : ", cohen_kappa_score(test_df['level'], test_df['predLevel'], weights="quadratic"))

[('0', 47846), ('1', 157), ('2', 4693), ('3', 680), ('4', 198)]
[('0', 39533), ('1', 3762), ('2', 7861), ('3', 1214), ('4', 1206)]
79.08205166492459
              precision    recall  f1-score   support

           0       0.82      0.99      0.89     39532
           1       0.30      0.01      0.02      3762
           2       0.61      0.36      0.45      7860
           3       0.31      0.17      0.22      1214
           4       0.90      0.15      0.25      1206

    accuracy                           0.79     53574
   macro avg       0.59      0.34      0.37     53574
weighted avg       0.74      0.79      0.74     53574

[[39092    45   359    24    12]
 [ 3432    47   273    10     0]
 [ 4675    64  2841   274     6]
 [  239     1   761   211     2]
 [  408     0   459   161   178]]
Test Accuracy score :  0.7908500391981185
Test Weighted F1 score :  0.7391025397859613
Test Micro F1 score :  0.7908500391981185
Test Precision score :  0.7399939267342663
Test Recall score :  0.7